## stratified continuous split to just split into 2:

In [1]:
# def stratified_continuous_split(X:pd.DataFrame, y:pd.Series, train_size:float, val_size:float, test_size:float, random_state:int):
#     '''
#     Performs a stratified split of inputted data (with respect to y) into 2 sets of specified percentages 
#     of the data using verstack's scsplit and performs basic error checking.

#     Parameters:
#     - X: a 2D pandas DataFrame, the feature matrix
#     - y: a 1D pandas Series, the target variable matrix matching X
#     - train_size: a float between 0 and 1, the percentage of X which should be training data
#     - test_size: a float between 0 and 1, the percentage of X which should be reserved for testing
#     - random_state: an int, the random state to split with
#     Note: The sum of train_size + test_size must be 1.0 (100% of X).

#     Returns:
#     - (X_train) a 2D pandas DataFrame, the feature matrix of training data
#     - (y_train) a 1D pandas Series, the target variable matrix for training data
#     - (X_test) a 2D pandas DataFrame, the feature matrix of testing data
#     - (y_test) a 1D pandas Series, the target variable matrix for testing data

#     Raises:
#     - ValueError for invalid input
#     '''
#     from verstack.stratified_continuous_split import scsplit
    
#     if ((train_size + val_size + test_size) != 1):
#         raise ValueError('Your train_size + test_size must add up to 1 (100%)!')
#     if (not isinstance(random_state, int)):
#         raise ValueError('Your random_state must be an int!')

#     X_train, X_other, y_train, y_other = scsplit(X, y, stratify=y, test_size=test_size, random_state=random_state)

#     # basic error checking to check that split returned train and test of expected sizes
#     train_count_low = (int)(train_size * X_len)
#     train_count_high = ceil(train_size * X_len)
#     test_count_low = (int)(test_size * X_len)
#     test_count_high = ceil(test_size * X_len)
    
#     Xtrain_fin = X_train.shape[0]
#     ytrain_fin = y_train.shape[0]
#     Xtest_fin = X_test.shape[0]
#     ytest_fin = y_test.shape[0]
    
#     if not (((Xtrain_fin == train_count_low) or (Xtrain_fin == train_count_high)) and ((ytrain_fin == train_count_low) or (ytrain_fin == train_count_high))):
#         raise ValueError(f'Training set size should be approx. {train_size * X_len}, instead is: {X_train.shape[0]}')
#     if not (((Xtest_fin == test_count_low) or (Xtest_fin == test_count_high)) and ((ytest_fin == test_count_low) or (ytest_fin == test_count_high))):
#         raise ValueError(f'Test set size should be approx. {test_size * X_len}, instead is: {X_test.shape[0]}')

#     return X_train, y_train, X_test, y_test

## ML pipeline with GridSearchCV, incomplete:

In [2]:
def MLpipe_Stratified_Continous_RMSE(X, y, preprocessor, ML_algo, param_grid):
    '''
    This function splits the data to train, validation, and test (60/20/20).
    The RMSE is minimized in cross-validation.
    
    This function:
    1. Loops through 10 different random states
    2. Splits the data 60/20/20.
    3. Fits a model using GridSearchCV with the predefined Preprocessor 
    4. Calculates the model's error on the test set 
    5. Returns a list of 10 test scores and 10 best models
    '''
    
    # lists to be returned
    test_scores = []
    best_models = []

    nr_states = 10
    for i in range(nr_states):
        rs = 28 * i
        print('Random State:', rs) # TEMPORARY

        # split
        X_train, y_train, X_val, y_val, X_test, y_test = stratified_continous_split(X, y, train_size=0.6, val_size=0.2, test_size=0.2, random_state=rs)
        
        # preprocess, train, and perform cross-validation
        pipe = make_pipeline(preprocessor, ML_algo)
        grid = GridSearchCV(pipe, param_grid=param_grid, scoring='neg_mean_squared_error', # using PredefinedSplit??????
                            cv=kf, return_train_score=True, n_jobs=-1, verbose=True)
        grid.fit(X_other, y_other)
        results = pd.DataFrame(grid.cv_results_)
        #print(results) # TEMPORARY

        # save results
        print('    Best Model Parameters:', grid.best_params_) # TEMPORARY
        print('    Validation RMSE Score:', grid.best_score_) # TEMPORARY
        best_models.append(grid)
        y_test_pred = grid.predict(X_test)
        test_score = mean_squared_error(y_test, y_test_pred, squared=False)
        test_scores.append(test_score)
        print('    Test RMSE Score:', test_score) # TEMPORARY
        
    return test_scores, best_models